In [1]:
from __future__ import print_function
from six.moves import cPickle as pickle
import tensorflow as tf

In [2]:
f=open('Data.pickle','rb')
Data=open('Data.pickle','rb')
train_dataset=pickle.load(Data)['train_dataset']
Data.seek(0)
train_labels=pickle.load(Data)['train_labels']
Data.seek(0)
test_dataset=pickle.load(Data)['test_dataset']
Data.seek(0)
test_labels=pickle.load(Data)['test_labels']
Data.close()

In [3]:
print(train_dataset.shape)

(50000, 28, 28)


In [4]:
train_dataset=train_dataset.reshape(train_dataset.shape[0],-1).T
train_labels=train_labels.reshape(train_labels.shape[0],-1).T
test_dataset=train_dataset.reshape(test_dataset.shape[0],-1).T
test_labels=train_labels.reshape(test_labels.shape[0],-1).T

In [5]:
def One_hot_encoder(labels,C):
    C=tf.constant(C,dtype=tf.int32,name='C')
    encoded=tf.one_hot(labels,C)
    with tf.Session() as sess:
        return sess.run(encoded)

In [6]:
Train_labels=One_hot_encoder(train_labels,10)
#rint(Train_labels.shape)
Train_labels=Train_labels[0]
print(Train_labels.shape)
#rint(train_labels[0])
#rint(Train_labels[0][0])
print(train_dataset.shape)

(50000, 10)
(784, 50000)


In [7]:
total_layers=9
Layer_Units={1:28,2:28,3:28,4:28,5:28,6:28,7:28,8:28}
Layer_Units[total_layers]=10
cache={}
for i in range(1,total_layers+1):
    cache['Z'+str(i)]=0
    cache['A'+str(i)]=0
parameters={}
count=1
for i in Layer_Units:
    num=i
    weight="W"+str(num)
    bias='b'+str(num)
    if i==1:
        parameters[weight]=tf.get_variable(weight,[Layer_Units[i],784],initializer=tf.contrib.layers.xavier_initializer(seed=1))
        parameters[bias] = tf.get_variable(bias, [Layer_Units[i], 1], initializer = tf.zeros_initializer())
    else:
        parameters[weight]=tf.get_variable(weight,[Layer_Units[i],Layer_Units[i-1]],initializer=tf.contrib.layers.xavier_initializer(seed=1)) 
        parameters[bias] = tf.get_variable(bias, [Layer_Units[i], 1], initializer = tf.zeros_initializer())

In [8]:
for i in Layer_Units:
        num=i
        if i==1:
            cache['Z'+str(i)]=tf.add(tf.matmul(parameters['W'+str(num)],train_dataset),parameters['b'+str(num)])
            cache['A'+str(i)]=tf.nn.relu(cache['Z'+str(i)])   
        elif i==total_layers:
            cache['Z'+str(i)]=tf.add(tf.matmul(parameters['W'+str(num)],cache['A'+str(i-1)]),parameters['b'+str(num)])
        else:
            cache['Z'+str(i)]=tf.add(tf.matmul(parameters['W'+str(num)],cache['A'+str(i-1)]),parameters['b'+str(num)])
            cache['A'+str(i)]=tf.nn.relu(cache['Z'+str(i)])

In [ ]:
logits = tf.transpose(cache['Z'+str(total_layers)])
labels = Train_labels
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels =labels))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
count=1
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()
with tf.Session() as session:
    session.run(init)
    print('Number','Loss','Accuracy')
    for i in range(1000):
        session.run(train_op)
        loss, acc = session.run([cost, accuracy])
        print(count,':',loss,':',acc)
        count=count+1   

Loss Accuracy
2.30146 : 0.10052
2.39961 : 0.09996
2.28404 : 0.13868
2.28781 : 0.17388
2.25385 : 0.13432
2.23689 : 0.13244
2.19157 : 0.13096
2.14068 : 0.16614
2.08 : 0.20132
2.0137 : 0.19424
1.97968 : 0.20626
2.10483 : 0.2082
1.88813 : 0.26484
1.96923 : 0.24688
1.91311 : 0.2553
1.94257 : 0.25364
1.81086 : 0.3155
1.88498 : 0.2653
1.79531 : 0.29024
1.81253 : 0.29574
1.77795 : 0.2878
1.76247 : 0.29376
1.71933 : 0.33978
1.73289 : 0.30528
1.67288 : 0.367
1.65786 : 0.35458
1.60756 : 0.38806
1.59143 : 0.41752
1.56127 : 0.41162
1.49886 : 0.45258
1.50036 : 0.46752
1.42273 : 0.49274
1.41861 : 0.48662
1.37087 : 0.50984
1.36784 : 0.50666
1.37941 : 0.50986
1.35857 : 0.5096
1.32837 : 0.51752
1.26167 : 0.54698
1.28254 : 0.53962
1.2436 : 0.55698
1.22677 : 0.5626
1.21538 : 0.56932
1.20277 : 0.57292
1.17295 : 0.58698
1.17015 : 0.5849
1.15448 : 0.5843
1.13588 : 0.5965
1.12425 : 0.60022
1.11155 : 0.60556
1.10155 : 0.61246
1.07883 : 0.62294
1.06771 : 0.62742
1.05738 : 0.63314
1.04948 : 0.6336
1.03655 : 0.64

0.468737 : 0.83708
0.468352 : 0.83724
0.467958 : 0.83728
0.467585 : 0.83732
0.467186 : 0.83742
0.466803 : 0.8373
0.466399 : 0.83762
0.465987 : 0.83774
0.465604 : 0.83772
0.465228 : 0.83784
0.464852 : 0.8379
0.464495 : 0.83834
0.464114 : 0.83828
0.463741 : 0.83858
0.463363 : 0.83852
0.462995 : 0.83868
0.462655 : 0.8389
0.462333 : 0.83886
0.461991 : 0.83918
0.461591 : 0.83922
0.46123 : 0.83936
0.460908 : 0.8394
0.460602 : 0.83952
0.460233 : 0.8397
0.459857 : 0.83962
0.459541 : 0.83992
0.459225 : 0.83996
0.458935 : 0.84008
0.458566 : 0.84006
0.458205 : 0.84024
0.457877 : 0.84056
0.457564 : 0.84056
0.457271 : 0.84056
0.456906 : 0.84084
0.456583 : 0.84062
0.456236 : 0.84104
0.455903 : 0.84102
0.455594 : 0.84096
0.455281 : 0.84112
0.454997 : 0.84102
0.45475 : 0.84118
0.454434 : 0.8413
0.454166 : 0.84146
0.453811 : 0.84126
0.453492 : 0.84136
0.453125 : 0.8418
0.452869 : 0.84182
0.452637 : 0.84198
0.452318 : 0.8417
0.451967 : 0.84212
0.451647 : 0.84216
0.451304 : 0.84242
0.451036 : 0.84236
0.4